# Miniproject Question 4 - Neuroscience: cellular and circuit mechanisms (BIO-482)

#### Importing libraries

In [49]:
import os
import sys
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

#### Importing helper functions
 Feel free to check out what these functions do in the corresponding files, `helpers.py` or `utils.py`. 
 - `helpers.py` contains functions to analyze membrane potential recording data
 - `utils.py` contains functions useful for other things e.g. plotting

In [21]:
base_path = os.getcwd()
base_path = base_path.replace('notebooks', 'scripts') # note: if you have notebooks twice in your base_path, this won't work
sys.path.insert(1, base_path)

from helpers import *
from utils import remove_top_right_frame, jitter_scatterplot

# Test

#### Load data

In [22]:
# Make paths
main_dir = r'/Users/family/Desktop/neurosciences-cell mechanisms/project/BIO482_MiniProject_2025_Python'
print('Main working directory:', main_dir)
figure_path = os.path.join(main_dir, 'Figures')
os.makedirs(figure_path, exist_ok=True)
data_path = os.path.join(main_dir, 'Data') #if your folder is organized differently, just specify the full path to the .mat file

fname = 'data_bio482.pkl'
data_df = pd.read_pickle(os.path.join(data_path, fname))

Main working directory: /Users/family/Desktop/neurosciences-cell mechanisms/project/BIO482_MiniProject_2025_Python


Inspect how the dataframe looks like:

In [23]:
data_df.head(5)

,Cell_APThreshold_Slope,Cell_Anatomy,Cell_Counter,Cell_Depth,Cell_ID,Cell_Layer,Cell_TargetedBrainArea,Cell_Type,Cell_tdTomatoExpressing,Mouse_DateOfBirth,...,Sweep_Counter,Sweep_MembranePotential,Sweep_MembranePotential_SamplingRate,Sweep_PassiveContactTimes,Sweep_QuietTimes,Sweep_StartTime,Sweep_Type,Sweep_WhiskerAngle,Sweep_WhiskerAngle_SamplingRate,Sweep_WhiskingTimes
0,10.0,L2/3;C2,1.0,229.0,SC901_1,L2/3,C2,EXC,False,NaN,...,1.0,"[-0.044009375, -0.044028125, -0.0439875, -0.04...",20000.0,"[[4.567, 4.713], [7.327, 7.519], [14.481, 14.6...","[[0.002, 4.564], [4.606, 6.09], [14.634, 15.52...",2005.0,active touch,"[2.740000000000009, 2.740000000000009, 2.74000...",100.0,"[[6.118, 6.326], [7.942, 13.856]]"
1,10.0,L2/3;C2,1.0,229.0,SC901_1,L2/3,C2,EXC,False,NaN,...,2.0,"[-0.046275, -0.04629375, -0.046259375, -0.0462...",20000.0,"[[15.274999999999999, 15.375], [16.247, 16.643...","[[0.002, 4.496], [5.558, 6.626], [13.406, 15.2...",2005.0,active touch,"[2.680000000000007, 2.680000000000007, 2.68000...",100.0,"[[4.7, 5.196], [6.696, 11.15], [12.032, 13.006..."
2,10.0,L2/3;C2,1.0,229.0,SC901_1,L2/3,C2,EXC,False,NaN,...,3.0,"[-0.041896875, -0.0419125, -0.041909375, -0.04...",20000.0,"[[4.347000000000001, 7.0390000000000015], [8.2...","[[0.002, 2.128], [6.252, 7.01], [7.15, 20.0]]",2005.0,passive contact,"[1.4958937492820894, 1.5019819900587095, 1.509...",100.0,"[[2.15, 3.652], [4.362, 5.848]]"
3,10.0,L2/3;C2,1.0,229.0,SC901_1,L2/3,C2,EXC,False,NaN,...,4.0,"[-0.04655625, -0.0465875, -0.046575, -0.046596...",20000.0,nan,"[[0.002, 2.764], [12.03, 14.464], [14.814, 15....",2005.0,active touch,"[2.467493802679826, 2.459711310964792, 2.46562...",100.0,"[[2.766, 12.026], [15.626, 18.406], [18.814, 1..."
4,10.0,L2/3;C2,1.0,229.0,SC901_1,L2/3,C2,EXC,False,NaN,...,5.0,"[-0.047296875, -0.047284375, -0.047265625, -0....",20000.0,"[[4.411000000000001, 4.4809999999999945], [5.2...","[[0.002, 3.36], [3.57, 4.424], [4.48, 5.224], ...",2005.0,active touch,"[2.4399999999999977, 2.4399999999999977, 2.440...",100.0,"[[5.228, 5.624], [7.462, 8.022], [11.002, 15.2..."


In [30]:
print(data_df.columns)

Index(['Cell_APThreshold_Slope', 'Cell_Anatomy', 'Cell_Counter', 'Cell_Depth',
       'Cell_ID', 'Cell_Layer', 'Cell_TargetedBrainArea', 'Cell_Type',
       'Cell_tdTomatoExpressing', 'Mouse_DateOfBirth', 'Mouse_Genotype',
       'Mouse_Name', 'Mouse_Sex', 'Sweep_ActiveContactTimes', 'Sweep_Counter',
       'Sweep_MembranePotential', 'Sweep_MembranePotential_SamplingRate',
       'Sweep_PassiveContactTimes', 'Sweep_QuietTimes', 'Sweep_StartTime',
       'Sweep_Type', 'Sweep_WhiskerAngle', 'Sweep_WhiskerAngle_SamplingRate',
       'Sweep_WhiskingTimes'],
      dtype='object')


In [24]:
sweep_type = 'free whisking'
time_window = 2    # time window to analyze Vm (s)
freq_band_lim = [1, 10, 30, 90] # low- and high-frequency band limits (Hz)

In [25]:
data_df_subset = data_df[data_df['Sweep_Type']==sweep_type] 

In [26]:
drop_cols = [
    'Cell_ID',
    'Cell_Counter',
    'Mouse_DateOfBirth',
    'Sweep_Counter'
]


In [11]:
class_color = {'EXC':'k',
               'PV':'indianred',
               'VIP':'royalblue',
               'SST':'darkorange'} 
cell_class_order = ['EXC', 'PV', 'VIP', 'SST']

In [42]:
df = data_df_subset.copy()

## Classifier

In [50]:

# --- 2. Funzione robusta per aggregare le serie temporali ---
def mean_std_series_robust(series):
    means = []
    stds = []

    for val in series:
        # Caso NaN o None
        if val is None or (isinstance(val, float) and np.isnan(val)):
            means.append(0)
            stds.append(0)
            continue

        # Se è stringa, prova a convertirla in lista
        if isinstance(val, str):
            try:
                val = eval(val)
            except:
                val = []

        # Se è NumPy array o lista annidata, appiattisci
        flat = []
        try:
            for item in val:
                if isinstance(item, (list, tuple, np.ndarray)):
                    flat.extend(item)
                else:
                    flat.append(item)
        except TypeError:
            # Se non è iterabile, usa il valore singolo
            flat = [val]

        # Calcola media e std
        if len(flat) == 0:
            means.append(0)
            stds.append(0)
        else:
            flat = [float(x) for x in flat]  # assicurati che siano numeri
            means.append(np.mean(flat))
            stds.append(np.std(flat))

    return means, stds


# --- 3. Costruzione delle feature ---
features = pd.DataFrame()

# AP Threshold
features['Cell_APThreshold_Slope'] = df['Cell_APThreshold_Slope'].fillna(0)

# Membrane potential
mp_mean, mp_std = mean_std_series_robust(df['Sweep_MembranePotential'])
features['Sweep_MembranePotential_mean'] = mp_mean
features['Sweep_MembranePotential_std'] = mp_std

# Passive / Active Contact Times
act_mean, act_std = mean_std_series_robust(df['Sweep_ActiveContactTimes'])
pass_mean, pass_std = mean_std_series_robust(df['Sweep_PassiveContactTimes'])
features['Sweep_ActiveContactTimes_mean'] = act_mean
features['Sweep_ActiveContactTimes_std'] = act_std
features['Sweep_PassiveContactTimes_mean'] = pass_mean
features['Sweep_PassiveContactTimes_std'] = pass_std

# Whisker Angle / WhiskingTimes
wa_mean, wa_std = mean_std_series_robust(df['Sweep_WhiskerAngle'])
wt_mean, wt_std = mean_std_series_robust(df['Sweep_WhiskingTimes'])
features['Sweep_WhiskerAngle_mean'] = wa_mean
features['Sweep_WhiskerAngle_std'] = wa_std
features['Sweep_WhiskingTimes_mean'] = wt_mean
features['Sweep_WhiskingTimes_std'] = wt_std

# Categorical features
for col in ['Cell_Type']:
    features[col] = LabelEncoder().fit_transform(df[col].fillna('NA'))


# --- 4. Target ---
y = LabelEncoder().fit_transform(df['Cell_Layer'])

# --- 5. Split train/test ---
X_train, X_test, y_train, y_test = train_test_split(
    features, y, test_size=0.2, random_state=42, stratify=y
)

# --- 6. Random Forest ---
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

# --- 7. Valutazione ---
score = clf.score(X_test, y_test)
cv_score = cross_val_score(clf, features, y, cv=5).mean()

print(f"Accuracy sul test set: {score:.3f}")
print(f"Accuracy media 5-fold CV: {cv_score:.3f}")

# --- 8. Feature importances ---
importances = pd.Series(clf.feature_importances_, index=features.columns).sort_values(ascending=False)
print("Top 10 feature più importanti:\n", importances.head(10))


Accuracy sul test set: 0.870
Accuracy media 5-fold CV: 0.745
Top 10 feature più importanti:
 Sweep_WhiskerAngle_mean          0.260517
Sweep_WhiskingTimes_std          0.150274
Sweep_WhiskingTimes_mean         0.125940
Sweep_MembranePotential_std      0.110560
Sweep_MembranePotential_mean     0.099373
Sweep_WhiskerAngle_std           0.099197
Cell_Type                        0.078552
Cell_APThreshold_Slope           0.075586
Sweep_ActiveContactTimes_mean    0.000000
Sweep_ActiveContactTimes_std     0.000000
dtype: float64


In [47]:
X_train.head()

,Cell_APThreshold_Slope,Sweep_MembranePotential_mean,Sweep_MembranePotential_std,Sweep_ActiveContactTimes_mean,Sweep_ActiveContactTimes_std,Sweep_PassiveContactTimes_mean,Sweep_PassiveContactTimes_std,Sweep_WhiskerAngle_mean,Sweep_WhiskerAngle_std,Sweep_WhiskingTimes_mean,Sweep_WhiskingTimes_std,Cell_Type,Cell_tdTomatoExpressing,Cell_Anatomy,Cell_Depth
577,15.0,-0.049487,0.008073,NaN,NaN,NaN,NaN,195.002015,5.466601,18.084800,9.260388,0,0,12,460.0
209,10.0,-0.044080,0.007680,NaN,NaN,NaN,NaN,211.306791,11.516616,28.408417,12.863464,3,1,12,550.0
233,10.0,-0.048450,0.006467,NaN,NaN,NaN,NaN,193.904548,6.609561,29.865412,16.356630,1,1,12,242.0
218,10.0,-0.051777,0.006104,NaN,NaN,NaN,NaN,197.714268,7.499960,13.144571,7.004243,2,1,12,496.0
145,10.0,-0.062555,0.004624,NaN,NaN,NaN,NaN,3.938523,11.317995,11.923600,5.237974,0,0,3,330.0
